Importing necessary libraries.

In [3]:
import numpy as np
import pandas as pd
import sqlite3
import datetime

Reading dataset from .sqlite file downloaded from https://www.kaggle.com/rtatman/188-million-us-wildfires

In [ ]:
eng = sqlite3.connect('FPA_FOD_20170508.sqlite')
query = 'SELECT * FROM Fires'

In [ ]:
df = pd.read_sql(query, eng)

Saving to .csv for easier access.

In [ ]:
df.to_csv('wildfires.csv')

Checking column for later dropping. Columns description can be found here: https://www.kaggle.com/rtatman/188-million-us-wildfires

In [4]:
df = pd.read_csv('wildfires.csv', low_memory=False, index_col=0)

/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [88]:
df.columns

Index(['id', 'year', 'disc_date', 'disc_doy', 'cause_code', 'cause', 'size',
       'size_class', 'lat', 'lon', 'state', 'shape'],
      dtype='object')

Dropping unneeded columns. The following attributes are not related to the location, date or size of the wildfire and therefore are not of interest for my analysis.

In [ ]:
columns_to_drop = ['FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 
                   'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME',
                   'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER',
                   'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIPS_CODE', 'FIPS_NAME', 'OWNER_CODE',
                   'OWNER_DESCR', 'COUNTY', 'Shape']

In [ ]:
df.drop(columns_to_drop, axis=1, inplace=True)

In [90]:
df.columns

Index(['id', 'year', 'disc_date', 'disc_doy', 'cause_code', 'cause', 'size',
       'size_class', 'lat', 'lon', 'state'],
      dtype='object')

Renaming and lowercasing columns for easier reading. 

In [42]:
df.columns = df.columns.str.lower()

In [51]:
df.rename(columns={'objectid': 'id', 'fire_year': 'year', 'discovery_date': 'disc_date', 'discovery_doy': 'disc_doy',
                   'discovery_time': 'disc_time', 'stat_cause_code': 'cause_code', 'stat_cause_descr': 'cause',
                   'fire_size': 'size', 'fire_size_class': 'size_class', 'latitude': 'lat', 'longitude': 'lon'},
                   inplace=True)

Checking NaNs.

In [79]:
df.isna().sum()

id                 0
year               0
disc_date          0
disc_doy           0
disc_time     882638
cause_code         0
cause              0
cont_date     891531
cont_doy      891531
cont_time     972553
size               0
size_class         0
lat                0
lon                0
state              0
shape              0
dtype: int64

Almost half of the observations are missing the time of discovery and control and the actual control date. Given this kind of scale I will not be basing any of my analysis on the time of day at which the wildfire took place or was controlled, and therefore I can simply drop the disc_time, cont_dat, cont_doy and cont_time columns.

In [82]:
df.drop(['disc_time', 'cont_date', 'cont_doy', 'cont_time'], axis=1, inplace=True)

Checking data types.

In [83]:
df.dtypes

id                     int64
year                   int64
disc_date     datetime64[ns]
disc_doy               int64
cause_code           float64
cause                 object
size                 float64
size_class            object
lat                  float64
lon                  float64
state                 object
shape                 object
dtype: object

The disc_date column can be converted to datetime format. Checking current format.

In [84]:
df.disc_date.head(1)

0   2005-02-02
Name: disc_date, dtype: datetime64[ns]

Julian date format.

In [55]:
epoch = pd.to_datetime(0, unit='s').to_julian_date()

In [57]:
df.disc_date = pd.to_datetime(df.disc_date - epoch, unit='D')

In [58]:
df.cont_date = pd.tallo_datetime(df.cont_date - epoch, unit='D')

Cleaning complete, the final look of the df is the following.

In [86]:
df.head()

,id,year,disc_date,disc_doy,cause_code,cause,size,size_class,lat,lon,state,shape
0,1,2005,2005-02-02,33,9.0,Miscellaneous,0.10,A,40.036944,-121.005833,CA,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2004,2004-05-12,133,1.0,Lightning,0.25,A,38.933056,-120.404444,CA,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,2004,2004-05-31,152,5.0,Debris Burning,0.10,A,38.984167,-120.735556,CA,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,2004,2004-06-28,180,1.0,Lightning,0.10,A,38.559167,-119.913333,CA,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,2004,2004-06-28,180,1.0,Lightning,0.10,A,38.559167,-119.933056,CA,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


Saving to new .csv for easier access.

In [91]:
df.to_csv('wildfires_clean.csv')

During the analysis I have found that there are 52 states in the list and not 50.

In [23]:
df = pd.read_csv('wildfires_clean.csv', index_col=0)

/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [24]:
sorted(df.state.unique())

['AK',
 'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'PR',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY']

The 2 excess states are DC (District of Columbia), which is actually a federal district, and PR (Puerto Rico), which is an unincorporated territory. I will be removing all rows related to the two from the dataframe and saving it again.

In [28]:
df = df[df.state != 'DC']

In [29]:
df = df[df.state != 'PR']

In [30]:
df.state.unique()

array(['CA', 'NM', 'OR', 'NC', 'WY', 'CO', 'WA', 'MT', 'UT', 'AZ', 'SD',
       'AR', 'NV', 'ID', 'MN', 'TX', 'FL', 'SC', 'LA', 'OK', 'KS', 'MO',
       'NE', 'MI', 'KY', 'OH', 'IN', 'VA', 'IL', 'TN', 'GA', 'AK', 'ND',
       'WV', 'WI', 'AL', 'NH', 'PA', 'MS', 'ME', 'VT', 'NY', 'IA', 'MD',
       'CT', 'MA', 'NJ', 'HI', 'DE', 'RI'], dtype=object)

In [31]:
df.to_csv('wildfires_clean.csv')